In [1]:
import torch
import torch.nn as nn
from torchtext import data
from torchtext import datasets

In [2]:
TEXT = data.Field(pad_first = True, fix_length=500)
LABEL = data.LabelField(dtype=torch.float)

In [5]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


.data\imdb\aclImdb_v1.tar.gz: 100%|███████████████████████████████████████████████| 84.1M/84.1M [00:53<00:00, 1.59MB/s]


In [9]:
TEXT.build_vocab(train_data, max_size = 50000)
LABEL.build_vocab(train_data)

In [10]:
vocab_size = len(TEXT.vocab)
n_classes = 2
print(vocab_size)

50002


In [11]:
# 데이터 분리
train_data, val_data = train_data.split(split_ratio=0.8)

In [12]:
train_iter, val_iter, test_iter = data.BucketIterator.splits(
(train_data, val_data, test_data), batch_size=64, shuffle=True, repeat=False
)

In [20]:
class RNN(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(RNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.rnn = nn.RNN(embed_dim, self.hidden_dim, num_layers=self.n_layers, batch_first=True),
        self.out = nn.Linear(self.hidden_dim, n_classes)
        
    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0)) # 첫 히든 스테이트 0 벡터로 초기화
        x,_ = self.rnn(x, h_0) # 
        h_t = x[:, -1, :] # (배치 크기, 히든 스테이트의 크기)의 텐서로 크기가 변경. 즉 마지막 히든 스테이트만 가져옴
        self.dropout(h_t)
        logit = self.out(h_t)
        return logit
    

    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [21]:
model = RNN(1, 256, vocab_size, 128, n_classes, 0.5).to('cuda:0')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [22]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to('cuda:0'), batch.label.to('cuda:0')
        y.data.sub_(1) # 레이블 값을 0과 1로 변환
        optimizer.zero_grad()
        
        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

In [23]:
def evaluate(model, val_iter):
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to('cuda:0'), batch.label.to('cuda:0')
        y.data.sub_(1)
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy= 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [24]:
best_val_loss = None
for e in range(1, 11):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)
    
    print('[Epoch: %d] val loss : %5.2f | val accuracy : %5.2f'%(e, val_loss,
                                                                val_accuracy))
    
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir('snapshot'):
            os.makedirs('snapshot')
        torch.save(model.state_dict(), './snapshot/txtclassification.pt')
        best_val_loss = val_loss

TypeError: 'tuple' object is not callable